In [ ]:
import pybamm
import scipy.io
import pandas as pd
import plotly.graph_objects as go
from pybamm_bayes.workflows.utils.parameter_sets.airbus_ncr import airbus_cell_parameters

pybamm.set_logging_level("SPAM")


In [ ]:
data_path = '/Panasonic 18650PF Data/Panasonic 18650PF Data/25degC/C20 OCV and 1C discharge tests_start_of_tests/05-08-17_13.26 C20 OCV Test_C20_25dC.mat'
mat = scipy.io.loadmat(data_path)
mdata = mat['meas']
ndata = {n: mdata[n][0,0].flatten() for n in mdata.dtype.names}
columns = [n for n, v in ndata.items()]
d=dict((c, ndata[c][0]) for c in columns)
df=pd.DataFrame.from_dict(ndata)

In [ ]:
params = airbus_cell_parameters()
model = pybamm.lithium_ion.SPMe()

In [ ]:
# create interpolant
current_interpolant = pybamm.Interpolant(df.Time.values, -1*df.Current.values, pybamm.t)

# set drive cycle
params["Current function [A]"] = current_interpolant
params["Lower voltage cut-off [V]"] = 2.5
sim = pybamm.Simulation(model, parameter_values=params)
solution = sim.solve(t_eval=df.Time.values, initial_soc=1)
solution.plot()

In [ ]:
# fig = go.Figure()
# sto = np.linspace(0,1, num=100)
# fig.add_trace(go.Scatter(x=sto, y = graphite_mcmb2528_ocp_Dualfoil1998(sto)))
# pos_ocp = params["Positive electrode OCP [V]"]
# fig.add_trace(go.Scatter(x=sto, y = pos_ocp(sto)))


In [ ]:
fig = go.Figure()
fig.add_trace(go.Scatter(x=df.Time, y = df.Voltage))
fig.add_trace(go.Scatter(x=solution.t, y = solution["Voltage [V]"].entries))

In [ ]:
fig.add_trace(go.Scatter(x=df.Time, y = df.Current))